# Tratamiento del fichero csv de cervezas obtenido de la web *soloartesanas.es*

In [1]:
import numpy as np
import pandas as pd
import yaml

#pd.options.display.max_colwidth = 300

In [2]:
# Importo el fichero "soloartesanas.csv"

In [3]:
df_soloartesanas=pd.read_csv("data/df_soloartesanas.csv", sep=";")
df_soloartesanas.head()

,Unnamed: 0,id,color,name,brand,rate_beer,features,description,image
0,0,1,rubia,"Arriaca Radler, cerveza artesana con limón (lata)",(cervezas artesanas Cervezas Arriaca),NaN,"{'Estilo de cerveza': ' Blonde Ale', 'Graduaci...",\nLa primera cerveza Radler artesana creada en...,https://static2.soloartesanas.es/3437-large_de...
1,1,2,rubia,Alegría & Boris Brew Ipanosuarus Rex,(cervezas artesanas Cervezas Alegría),NaN,{'Estilo de cerveza': ' Imperial / Double IPA'...,"\nUna doble IPA rubia, potente y seca. Elabora...",https://static3.soloartesanas.es/3333-large_de...
2,2,3,rubia,Roy The Bull Mango,(cervezas artesanas Roy The Bull),NaN,"{'Estilo de cerveza': ' Fruit Beer', 'Graduaci...",\nAquí está la primera cerveza de Roy The Bull...,https://static3.soloartesanas.es/3332-large_de...
3,3,4,rubia,Marijuana,(cervezas artesanas Marijuana),NaN,"{'Estilo de cerveza': ' Pale Ale', 'Graduación...",\nLa única cerveza cerveza artesana que podrás...,https://static1.soloartesanas.es/3223-large_de...
4,4,5,rubia,Arriaca Trigo (lata),(cervezas artesanas Cervezas Arriaca),NaN,"{'Estilo de cerveza': ' Weizen - Weissbier', '...",\nUna cerveza de trigo muy afrutada y ligera. ...,https://static2.soloartesanas.es/3449-large_de...


In [4]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   435 non-null    int64 
 1   id           435 non-null    int64 
 2   color        435 non-null    object
 3   name         435 non-null    object
 4   brand        435 non-null    object
 5   rate_beer    157 non-null    object
 6   features     435 non-null    object
 7   description  433 non-null    object
 8   image        435 non-null    object
dtypes: int64(2), object(7)
memory usage: 30.7+ KB


## Limpieza inicial del fichero
Revisión inicial del dataframe para eliminar columnas inncecesarias, agrupar información, unificar tipos y contenidos, etc. de modo que se pueda realizar después un análisis de la información.

Para empezar, gran parte de la información está recogida en el campo "features" que tiene estructura de diccionario y al que habrá que extraer la información separándola en columnas.

In [5]:
# Veo cómo es el campo "features" con un ejemplo
df_soloartesanas.iloc[355, :]['features']

"{'Estilo de cerveza': ' Stout', 'Graduación alcohólica': ' 4,2% Alc./Vol.', 'Amargor (según la Unidad Internacional de Amargor)': ' 40 IBUs', 'Color': 'Negra', 'Grano': 'Avena', 'Fermentación': 'Ale', 'European Brewing Convention (EBC)': '110', 'Maltas': 'Maris Otter Floor Malt, Chocolate, Roasted Barley, Crystal y Avena', 'Lúpulos': 'Cascade', 'Población': 'Liérganes', 'Provincia': 'Cantabria', 'Comunidad Autónoma': 'Cantabria', 'País': 'España'}"

In [6]:
# para ver las características del registro más largo

df_soloartesanas["num_caract"]= df_soloartesanas['features'].apply(len)

features_mas_largo=df_soloartesanas[df_soloartesanas["num_caract"]==df_soloartesanas["num_caract"].max()]['features']
df_soloartesanas.drop(columns="num_caract",inplace=True)

features_mas_largo.values

array(["{'Estilo de cerveza': ' Amber Ale', 'Graduación alcohólica': ' 5.3% Alc./Vol.', 'Amargor (según la Unidad Internacional de Amargor)': ' 30 IBUs', 'Color': 'Tostada', 'Grano': 'Cebada', 'Fermentación': 'Ale', 'European Brewing Convention (EBC)': '40', 'Maltas': 'Pale Ale, Crystal, Victoria y CaraAroma', 'Lúpulos': 'Kent Golding y Fuggles', 'Población': 'Oviedo', 'Provincia': 'Asturias', 'Comunidad Autónoma': 'Asturias', 'País': 'España', 'Temperatura de consumo': '10-13ºC', 'Vaso recomendado': 'Vaso ancho'}"],
      dtype=object)

In [7]:
# Hay que transformar la columna de features, que es un diccionario, en varias columnas 
df_soloartesanas['features']

0      {'Estilo de cerveza': ' Blonde Ale', 'Graduaci...
1      {'Estilo de cerveza': ' Imperial / Double IPA'...
2      {'Estilo de cerveza': ' Fruit Beer', 'Graduaci...
3      {'Estilo de cerveza': ' Pale Ale', 'Graduación...
4      {'Estilo de cerveza': ' Weizen - Weissbier', '...
                             ...                        
430    {'Estilo de cerveza': ' English Pale Ale', 'Gr...
431    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
432    {'Estilo de cerveza': ' Irish Red Ale', 'Gradu...
433    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
434    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
Name: features, Length: 435, dtype: object

In [8]:
# aplicar yaml a columna features y convertirla en diccionario aplicando map
# Para un registro sería: mi_dict2 = yaml.load(df_soloartesanas.features[0])

df_soloartesanas['features'] = df_soloartesanas.features.map(lambda x: yaml.load(x))

<ipython-input-8-f3f490e69a13>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  df_soloartesanas['features'] = df_soloartesanas.features.map(lambda x: yaml.load(x))


In [9]:
df_soloartesanas['features']

0      {'Estilo de cerveza': ' Blonde Ale', 'Graduaci...
1      {'Estilo de cerveza': ' Imperial / Double IPA'...
2      {'Estilo de cerveza': ' Fruit Beer', 'Graduaci...
3      {'Estilo de cerveza': ' Pale Ale', 'Graduación...
4      {'Estilo de cerveza': ' Weizen - Weissbier', '...
                             ...                        
430    {'Estilo de cerveza': ' English Pale Ale', 'Gr...
431    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
432    {'Estilo de cerveza': ' Irish Red Ale', 'Gradu...
433    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
434    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
Name: features, Length: 435, dtype: object

In [10]:
# veo un ejemplo
print(df_soloartesanas.iloc[355, :])

Unnamed: 0                                                   355
id                                                           356
color                                                      negra
name                                     Dougall's Session Stout
brand                             (cervezas artesanas Dougall's)
rate_beer                                               92 / 100
features       {'Estilo de cerveza': ' Stout', 'Graduación al...
description    \nDougalls Session Stout es una excelente cerv...
image          https://static3.soloartesanas.es/2095-large_de...
Name: 355, dtype: object


In [11]:
# Separo en columnas cada clave/valor del dicc de features

df_soloartesanas = pd.concat([df_soloartesanas.drop(['features'], axis=1), df_soloartesanas['features'].apply(pd.Series)], axis=1)

In [12]:
df_soloartesanas.head(2)

,Unnamed: 0,id,color,name,brand,rate_beer,description,image,Estilo de cerveza,Graduación alcohólica,...,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Vaso recomendado,Grano,Sin gluten,Ecológica,País
0,0,1,rubia,"Arriaca Radler, cerveza artesana con limón (lata)",(cervezas artesanas Cervezas Arriaca),NaN,\nLa primera cerveza Radler artesana creada en...,https://static2.soloartesanas.es/3437-large_de...,Blonde Ale,"2,7% Alc./Vol.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,rubia,Alegría & Boris Brew Ipanosuarus Rex,(cervezas artesanas Cervezas Alegría),NaN,"\nUna doble IPA rubia, potente y seca. Elabora...",https://static3.soloartesanas.es/3333-large_de...,Imperial / Double IPA,"8,2% Alc./Vol.",...,Valencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Compruebo las nuevas columnas de features, tus tipos y misings
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Unnamed: 0                                          435 non-null    int64 
 1   id                                                  435 non-null    int64 
 2   color                                               435 non-null    object
 3   name                                                435 non-null    object
 4   brand                                               435 non-null    object
 5   rate_beer                                           157 non-null    object
 6   description                                         433 non-null    object
 7   image                                               435 non-null    object
 8   Estilo de cerveza                                   419 non-null    object
 9   Graduación

Algunas de estas nuevas columnas las podré borrar si no aportan información porque falten muchos valores. En otras ocasiones, en las que se puedan deducir los datos o sean relativamente pocos como para poder imputar valores medios o modas, se podrán dejar esas características.

In [14]:
'''
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Unnamed: 0                                          435 non-null    int64 -- Borrar
 1   id                                                  435 non-null    int64 
 2   color                                               435 non-null    object
 3   name                                                435 non-null    object
 4   brand                                               435 non-null    object
 5   rate_beer                                           157 non-null    object -- Borrar
 6   description                                         433 non-null    object -- completar
 7   image                                               435 non-null    object
 8   Estilo de cerveza                                   419 non-null    object
 9   Graduación alcohólica                               415 non-null    object
 10  Color                                               418 non-null    object -- Borrar (duplicada)
 11  Fermentación                                        399 non-null    object
 12  Amargor (según la Unidad Internacional de Amargor)  275 non-null    object
 13  Población                                           409 non-null    object -- completar
 14  Provincia                                           426 non-null    object -- completar
 15  Comunidad Autónoma                                  424 non-null    object -- completar
 16  Lúpulos                                             99 non-null     object
 17  European Brewing Convention (EBC)                   101 non-null    object
 18  Maltas                                              87 non-null     object
 19  Temperatura de consumo                              137 non-null    object
 20  Vaso recomendado                                    52 non-null     object -- Borrar/añadir a descr
 21  Grano                                               310 non-null    object -- Analizar antes de decidir
 22  Sin gluten                                          2 non-null      object -- Borrar
 23  Ecológica                                           3 non-null      object -- Borrar
 24  País                                                45 non-null     object -- completar
dtypes: int64(2), object(23)

'''
print(" ")

In [15]:
# borro las columnas que no aportan valor por estar repetidas o tener muy pocos datos: 
# "Unnamed: 0","Vaso recomendado","Sin gluten","Ecológica" y "Color"

df_soloartesanas.drop(columns=["Unnamed: 0","rate_beer","Color","Vaso recomendado","Sin gluten","Ecológica"], inplace=True)

In [16]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 19 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   id                                                  435 non-null    int64 
 1   color                                               435 non-null    object
 2   name                                                435 non-null    object
 3   brand                                               435 non-null    object
 4   description                                         433 non-null    object
 5   image                                               435 non-null    object
 6   Estilo de cerveza                                   419 non-null    object
 7   Graduación alcohólica                               415 non-null    object
 8   Fermentación                                        399 non-null    object
 9   Amargor (s

In [17]:
# veo qué cervezas están sin descripción
df_soloartesanas[df_soloartesanas["description"].isna()]

,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País
171,172,rubia,Catalina,(cervezas artesanas Torquemada),NaN,https://static2.soloartesanas.es/886-large_def...,Pilsner,"4,8% Alc./Vol.",Ale,NaN,Torquemada,Palencia,Castilla y León,NaN,NaN,NaN,2-4ºC,Cebada,NaN
223,224,rubia,Illusió,(cervezas artesanas Ginesart),NaN,https://static2.soloartesanas.es/2664-large_de...,Iber Ale,"5,0% Alc./Vol.",Ale,NaN,Ginesart,Tarragona,Cataluña,Cascade y Saaz. Salvo,NaN,"Pale ale, Pilsen y de trigo",8-12 ºC,Trigo,NaN


In [18]:
# Añado una descripción de las cervezas que faltan y que encuentro en otras webs
df_soloartesanas.loc[171,"description"]="Se trata de una cerveza rubia tipo Pils, ligera y refrescante a base de malta Pilsner y lúpulos alemanes y checos. Catalina debe su nombre al nacimiento, el 14 de enero de 1507, de la Infanta Catalina de Austria (reina de Portugal) en la localidad Torquemada. Hija de Juana de Castilla y Felipe de Habsburgo. La cerveza Catalina es una cerveza rubia brillante, ligeramente amarga, con olores a cereal y una espuma equilibrada. El contenido alcohólico es de 4,8% Vol. lo que hace que esta cerveza sea la más ligera de toda la gama. Temperatura de consumo recomendada: 2º-4º"
df_soloartesanas.loc[223,"description"]="Estilo cervecero: Iber Ale. Cerveza rubia de alta fermentación adaptada al estilo de nuestro país. Nota de cata: Toda la intensidad del lúpulo en combinación con la malta, para hacer pasar la sed. Ingredientes: Malta Pale ale, Pilsen y de trigo. de los lúpulos: Cascade y Saaz. Levadura. Temperatura de consumo: entre 8 y 12 º C Graduación alcohólica: 5% Alc. Vol."

In [19]:
df_soloartesanas.iloc[223]["description"]

'Estilo cervecero: Iber Ale. Cerveza rubia de alta fermentación adaptada al estilo de nuestro país. Nota de cata: Toda la intensidad del lúpulo en combinación con la malta, para hacer pasar la sed. Ingredientes: Malta Pale ale, Pilsen y de trigo. de los lúpulos: Cascade y Saaz. Levadura. Temperatura de consumo: entre 8 y 12 º C Graduación alcohólica: 5% Alc. Vol.'

## Unificación la información sobre el origen de fabricación
Por la propia página web sabemos que todas las cervezas son de origen español, pero revisamos los datos para ver si falta información y si hay uniformidad en la nomenclaruta.

In [20]:
# compruebo que efectivamente solo hay cervezas españolas
print("Sin país: ", df_soloartesanas["País"].isna().sum())
df_soloartesanas.groupby("País")["id"].nunique()

Sin país:  390


País
ESpaña     1
España    44
Name: id, dtype: int64

Hay demasiadas cervezas sin identifiador de país. Puedo forzar la columna y poner en todas "España", pero antes voy a comprobar los otros campos sobre población, provincia y comunidad autónoma, para asegurarme de que no hay cervezas de otros.

Del total de 435 tenemos datos sobre:
- Población:                                           409 
- Provincia:                                           426 
- Comunidad Autónoma:                                 424 



In [21]:
# Compruebo las provincias que aparecen
df_soloartesanas["Provincia"].unique()

array([nan, 'Valencia', 'Pontevedra', 'Ávila', 'Guadalajara', 'Barcelona',
       'Segovia', 'Alicante', 'Burgos', 'La Rioja', 'Vizcaya', 'Madrid',
       'Córdoba', 'Málaga', 'Teruel', 'Valladolid', 'Badajoz',
       'Castellón', 'Cuenca', 'Tarragona', 'Girona', 'Álava', 'Albacete',
       'Soria', 'Lugo', 'Navarra', 'León', 'Palencia', 'Murcia', 'Huelva',
       'Asturias', 'Toledo', 'Cantabria', 'Sevilla', 'Ciudad Real',
       'Orense', 'La Coruña', 'Huesca', 'Lérida', 'Cádiz', 'Granada',
       'Guipúzcoa', 'Zaragoza', 'Zamora', 'Salamanca', 'Cáceres'],
      dtype=object)

In [22]:
# Estas son las CC.AA que aparecen
df_soloartesanas["Comunidad Autónoma"].unique()

array([nan, 'Valencia', 'Galicia', 'Castilla y León',
       'Castilla-La Mancha', 'Cataluña', 'La Rioja', 'País Vasco',
       'Madrid', 'Andalucía', 'Aragón', 'Extremadura', 'Navarra',
       'Murcia', 'Asturias', 'Canarias', 'Cantabria'], dtype=object)

In [23]:
# Durante las pruebas he comprobado que hay un error en la asignación de la C.A de Canarias
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"]=="Canarias"]

,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País
95,96,rubia,Smach Pale Ale,(cervezas artesanas Smach),\nSi lo que quieres es disfrutar de una cervez...,https://static1.soloartesanas.es/1722-large_de...,Pale Ale,"5,0% Alc./Vol.",Ale,25 IBUs,Camargo,Cantabria,Canarias,NaN,NaN,NaN,NaN,Cebada,España
97,98,rubia,Dougall's Happy Otter,(cervezas artesanas Dougall's),\nCerveza de temporada de Dougalls. Una Pale A...,https://static3.soloartesanas.es/1719-large_de...,Pale Ale,NaN,Ale,NaN,NaN,Cantabria,Canarias,NaN,NaN,NaN,NaN,Cebada,NaN


In [24]:
# Corrijo el dato 
df_soloartesanas.loc[df_soloartesanas["Comunidad Autónoma"]=="Canarias","Comunidad Autónoma"]='Cantabria'

In [25]:
# ¿Que CCAA faltan de las que sí sabemos la provincia?
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Provincia"]


0         NaN
29        NaN
30        NaN
41        NaN
63     Girona
69        NaN
230       NaN
351    Girona
352    Girona
372       NaN
396       NaN
Name: Provincia, dtype: object

In [26]:
# como son todas de Girona, actualizamos la CCAA con Cataluña
mask=(df_soloartesanas["Comunidad Autónoma"].isna()) & (df_soloartesanas["Provincia"]=="Girona")
df_soloartesanas.loc[mask,"Comunidad Autónoma"]="Cataluña"

In [27]:
# comprobamos que se han actualizado bien
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Provincia"]

0      NaN
29     NaN
30     NaN
41     NaN
69     NaN
230    NaN
372    NaN
396    NaN
Name: Provincia, dtype: object

In [28]:
# A la inversa, en las provincias faltan ¿cuál es la CC.AA.?
df_soloartesanas[df_soloartesanas["Provincia"].isna()]["Comunidad Autónoma"]

0           NaN
29          NaN
30          NaN
41          NaN
69          NaN
230         NaN
257    Cataluña
372         NaN
396         NaN
Name: Comunidad Autónoma, dtype: object

Comprobamos que las 8 provincias que faltan no tienen información sobre CC.AA, salvo 1 que es de Cataluña.
El siguiente paso será averiguar qué información tenemos y falta respecto a la población de origen, para ver si se pueden completar las otras columnas.

In [29]:
# Por un lado, en las provincias que faltan ¿cuál es población?
df_soloartesanas[df_soloartesanas["Provincia"].isna()]["Población"]

0             NaN
29            NaN
30            NaN
41            NaN
69            NaN
230           NaN
257    Granollers
372           NaN
396           NaN
Name: Población, dtype: object

In [30]:
# Lo mismo para las CC.AA.
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Población"]

0      NaN
29     NaN
30     NaN
41     NaN
69     NaN
230    NaN
372    NaN
396    NaN
Name: Población, dtype: object

Con la información de la columna "Población" solamente podemos completar los datos del registro 257 (Granollers), que corresponden a la provincia de Barcelona en Cataluña.

In [31]:
# Actualizo el registro 257 (Granollers)
df_soloartesanas.loc[257,"Provincia"]="Barcelona"

In [32]:
print("Cervezas con población de origen",df_soloartesanas["Población"].count())
print("Cervezas con provincia",df_soloartesanas["Provincia"].count())
print("Cervezas con CC.AA",df_soloartesanas["Comunidad Autónoma"].count())


Cervezas con población de origen 409
Cervezas con provincia 427
Cervezas con CC.AA 427


Esto significa que de las 435 cervezas recogidas 409 tienen toda la información, 8 no tienen ningún dato sobre procedencia y 18 tienen información regional.
Dada esta circunstancia, se decide:
1. Eliminar los 8 registros con origen totalmente desconocido.
2. Conservar los 18 incompletos, indicando que la población de origen está "sin especificar". 
3. Unificar como país España

Con ello, conseguiríamos un dataset con 427 registros documentados.

In [33]:
# Elimino los 8 registros sin ninguna información sobre su origien
mask=df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]
df_soloartesanas=df_soloartesanas.drop(mask.index)


In [34]:
# ¿Cuáles son las poblaciones que faltan?
df_soloartesanas[df_soloartesanas["Población"].isna()]["Provincia"]

18        Valencia
27          Madrid
39      Valladolid
40      Valladolid
88         Navarra
94       Barcelona
97       Cantabria
172    Guadalajara
173    Guadalajara
263        Vizcaya
283         Madrid
347         Girona
353    Guadalajara
386    Guadalajara
417     Valladolid
419      Barcelona
421      Salamanca
428    Guadalajara
Name: Provincia, dtype: object

In [35]:
# guardo las provincias de las poblaciones que faltan
prov_para_pobl=df_soloartesanas[df_soloartesanas["Población"].isna()]["Provincia"].unique()
prov_para_pobl

array(['Valencia', 'Madrid', 'Valladolid', 'Navarra', 'Barcelona',
       'Cantabria', 'Guadalajara', 'Vizcaya', 'Girona', 'Salamanca'],
      dtype=object)

In [36]:
# ejemplo para comprobar
mask_prov_ej=(df_soloartesanas["Provincia"]=='Valencia') 
mask_sin_pobl_ej=mask_prov_ej & (df_soloartesanas["Población"].isna())
df_soloartesanas[mask_sin_pobl_ej]

,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País
18,19,rubia,Birra & Blues Mago de Oz,(cervezas artesanas Birra & Blues),\nLa banda de rock Mago de Oz ya tiene su prop...,https://static1.soloartesanas.es/3014-large_de...,Blonde Ale,"4,5% Alc./Vol.",Ale,21 IBUs,NaN,Valencia,Valencia,NaN,7,NaN,NaN,NaN,NaN


In [37]:
prov_moda_ej=df_soloartesanas[mask_prov_ej]["Población"].mode()
prov_moda_ej

0    Massalfassar
dtype: object

In [38]:
# Para completar el dato de población, les voy a asignar por defecto
# la población que sea la moda en esa provincia

for provincia in prov_para_pobl:
    mask_prov=(df_soloartesanas["Provincia"]==provincia) 
    mask_sin_pobl=mask_prov & (df_soloartesanas["Población"].isna())
    prov_moda=df_soloartesanas[mask_prov]["Población"].mode()
    
    df_soloartesanas.loc[mask_sin_pobl,"Población"]=prov_moda[0]

# comprobamos
df_soloartesanas[df_soloartesanas["Población"].isna()]


,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País


In [39]:
# ejemplo para comprobar
df_soloartesanas.loc[18,"Población"]

'Massalfassar'

In [40]:
# Asigno "sin especificar" a las poblaciones que faltan pero sí conocemos su provincia y CC.AA
df_soloartesanas["Población"]=df_soloartesanas["Población"].fillna("sin especificar")

In [41]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427 entries, 1 to 434
Data columns (total 19 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   id                                                  427 non-null    int64 
 1   color                                               427 non-null    object
 2   name                                                427 non-null    object
 3   brand                                               427 non-null    object
 4   description                                         427 non-null    object
 5   image                                               427 non-null    object
 6   Estilo de cerveza                                   416 non-null    object
 7   Graduación alcohólica                               412 non-null    object
 8   Fermentación                                        396 non-null    object
 9   Amargor (s

In [42]:
df_soloartesanas.columns

Index(['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Población',
       'Provincia', 'Comunidad Autónoma', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'País'],
      dtype='object')

In [43]:
# Unifico el campo "País" que tiene Nan y distintas grafías
df_soloartesanas["País"]="España"

# Muevo las columnas para que la información sobre la procedencia esté ordenada y al final del dataset
df_soloartesanas=df_soloartesanas[['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población',
       'Provincia', 'Comunidad Autónoma', 'País']]


## Normalización de datos específicos sobre cervezas
A continuación se procede a revisar las columnas con información específica sobre las características de las cervezas, con el objetivo de que los datos sean uniformes y los tipos se correspondan con ellos. También se cambiarán los nombres de algunas columnas que resultan demasiado largos para trabajar.

In [44]:
df_soloartesanas.columns

Index(['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población', 'Provincia', 'Comunidad Autónoma', 'País'],
      dtype='object')

In [45]:
# cambio nombres de columnas demasiado largos
df_soloartesanas.rename(columns={'Estilo de cerveza':"Estilo",'Graduación alcohólica':"Graduación",'Amargor (según la Unidad Internacional de Amargor)':"IBU", },inplace=True)

### Datos sobre el estilo de las cervezas
Uno de los datos más importantes en la valoración de las cervezas es su estilo. Encontramos que este campo tiene algunos registros sin especificar, por lo que buscamos la manera de completar la información basándonos en otros datos o, de manera puntual, buscando el estilo para unos casos concretos.



In [46]:
df_soloartesanas["Estilo"].value_counts()

 Pale Ale                   34
 India Pale Ale - IPA       19
 Brown Ale                  17
 Blonde Ale                 14
 American Pale Ale - APA    11
                            ..
 IPA Single Hop Motueka      1
 Farmhouse Ale               1
 Pumpkin Ale                 1
 Red IPA                     1
 Bohemian Amber Ale          1
Name: Estilo, Length: 152, dtype: int64

In [47]:
df_soloartesanas["Estilo"].unique()

array([' Imperial / Double IPA', ' Fruit Beer', ' Pale Ale',
       ' Weizen - Weissbier', ' White IPA', ' Kölsch',
       ' English Pale Ale', ' Lager', ' Pumpkin Ale',
       ' India Pale Ale - IPA', ' Session IPA', ' Blonde Ale',
       ' Golden Ale', ' Witbier (Belgian White)', ' Amber Ale',
       ' American Pale Ale - APA', ' Smoked Beer', ' American Amber Ale',
       ' English IPA', ' Tripel', ' Belgian Blonde Ale',
       ' Golden Braggot', ' American Golden Ale', ' Double Red IPA',
       ' American IPA', ' Saison', ' Extra Pale Ale', ' Braggot',
       ' German Ale', ' Rauchbier', ' Belgian Dubbel', ' Belgian IPA',
       ' Cream Ale', ' Strong Ale', nan, ' Double IPA',
       ' California Common / Steam Beer', ' Farmhouse Ale',
       ' American Wheat', ' Spice / Herb / Vegetable',
       ' Pilsner - Pilsen', ' Summer Ale', ' India Pale Ale',
       ' Pilsner Ale', ' Belgian Ale', ' Biére de Garde',
       ' Wheat Ale Ecológica', ' Marzenbier', ' IPA con algarroba',
       

In [48]:
# ¿cómo son las que no tienen estilo?
df_estilo=df_soloartesanas[df_soloartesanas["Estilo"].isna()]

df_estilo=df_estilo.drop(columns=["id", "name","Graduación", "IBU","Lúpulos",
                                  "European Brewing Convention (EBC)","Maltas",
                                  'image',"brand","description",
                                 'Población', 'Provincia',
                                 'Comunidad Autónoma',
                                 'Temperatura de consumo', 'País'])
df_estilo

,color,Estilo,Fermentación,Grano
85,rubia,NaN,Ale,Cebada
172,rubia,NaN,Ale,Trigo
173,rubia,NaN,Ale,Cebada
184,rubia,NaN,Ale,Trigo
185,rubia,NaN,Ale,Cebada
190,rubia,NaN,Ale,Cebada
279,tostada,NaN,Ale,Trigo
286,tostada,NaN,Ale,Cebada
386,negra,NaN,Ale,Cebada
389,negra,NaN,Ale,Cebada


Observamos que para las 11 cervezas que no tienen clasificación de estilo podemos obtener el dato conociendo la fermentación, el color y el grano. 
Por tanto, prodecemos a obtener el estilo que más se repite en cada una de esas combinaciones y se lo añadimos a la cerveza. 

In [49]:
# Eliminamos las combinaciones que se repiten
df_estilo.drop_duplicates(inplace=True)

#inicializamos el índice
df_estilo.reset_index(drop=True,inplace=True)
df_estilo

,color,Estilo,Fermentación,Grano
0,rubia,NaN,Ale,Cebada
1,rubia,NaN,Ale,Trigo
2,tostada,NaN,Ale,Trigo
3,tostada,NaN,Ale,Cebada
4,negra,NaN,Ale,Cebada
5,roja,NaN,Ale,Cebada


In [50]:
# Buscamos el estilo que es la moda para cada combinación (ej.rubia & Ale & Cebada) y lo asignamos a cada cerveza sin estilo

combinaciones=range(df_estilo.columns.size+1)

for comb in combinaciones:
    
    mask=(df_soloartesanas["color"]==df_estilo.loc[comb,"color"])&(df_soloartesanas["Fermentación"]==df_estilo.loc[comb,"Fermentación"])&(df_soloartesanas["Grano"]==df_estilo.loc[comb,"Grano"])
    mk_sin_estilo=mask & df_soloartesanas["Estilo"].isna()

    nuevo_estilo=df_soloartesanas[mask]["Estilo"].mode()
    
    df_soloartesanas.loc[mk_sin_estilo,"Estilo"]=nuevo_estilo[0]

# comprobamos
df_soloartesanas[df_soloartesanas["Estilo"].isna()]



,id,color,name,brand,description,image,Estilo,Graduación,Fermentación,IBU,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,Población,Provincia,Comunidad Autónoma,País
428,429,roja,Arriaca IPA,(cervezas artesanas Cervezas Arriaca),\nArrica IPA es una de las cervezas artesanas ...,https://static2.soloartesanas.es/3441-large_de...,NaN,NaN,Ale,NaN,NaN,NaN,NaN,NaN,Cebada,Yunquera de Henares,Guadalajara,Castilla-La Mancha,España


In [51]:
# queda 1 cerveza sin estilo. Vemos su descripción para buscar más información
df_soloartesanas.loc[428,"description"]

'\nArrica IPA\xa0es una de las\xa0cervezas artesanas que elabora cervezas arriaca desde Yunquera de Henares, Guadalajara.\xa0Es una cerveza artesana IPA, Ale, con marcado carácter a lúpulo. \nDestaca su color\xa0color anaranjado fuerte tirando a rojizo, con una corona de espuma cremosa y de buena retención. Aromas cítricos, sobre todo a pomelo y mandarina, notas herbáceas, florales y matices afrutados.\xa0En boca presenta un amargor medio que se disipa rápidamente dejando paso al cítrico, al pino y al caramelo.\n¡Salud!\n'

In [52]:
# Por la descripción encajaría en una Red IPA. Actualizamos el dato
df_soloartesanas.loc[428,"Estilo"]=' Red IPA'

In [53]:
# Comprobamos
df_soloartesanas["Estilo"].unique()

array([' Imperial / Double IPA', ' Fruit Beer', ' Pale Ale',
       ' Weizen - Weissbier', ' White IPA', ' Kölsch',
       ' English Pale Ale', ' Lager', ' Pumpkin Ale',
       ' India Pale Ale - IPA', ' Session IPA', ' Blonde Ale',
       ' Golden Ale', ' Witbier (Belgian White)', ' Amber Ale',
       ' American Pale Ale - APA', ' Smoked Beer', ' American Amber Ale',
       ' English IPA', ' Tripel', ' Belgian Blonde Ale',
       ' Golden Braggot', ' American Golden Ale', ' Double Red IPA',
       ' American IPA', ' Saison', ' Extra Pale Ale', ' Braggot',
       ' German Ale', ' Rauchbier', ' Belgian Dubbel', ' Belgian IPA',
       ' Cream Ale', ' Strong Ale', ' Double IPA',
       ' California Common / Steam Beer', ' Farmhouse Ale',
       ' American Wheat', ' Spice / Herb / Vegetable',
       ' Pilsner - Pilsen', ' Summer Ale', ' India Pale Ale',
       ' Pilsner Ale', ' Belgian Ale', ' Biére de Garde',
       ' Wheat Ale Ecológica', ' Marzenbier', ' IPA con algarroba',
       ' Ste

In [54]:
# hay varias cervezas del mismo tipo con distinta nomenclarura
#df_soloartesanas[df_soloartesanas["Estilo"].str.contains("India Pale Ale")]["Estilo"].unique()
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Pale Ale")]["Estilo"].unique()

array([' Pale Ale', ' English Pale Ale', ' India Pale Ale - IPA',
       ' American Pale Ale - APA', ' Extra Pale Ale', ' India Pale Ale',
       ' India Pale Ale (IPA)', ' American Pale Ale', ' America Pale Ale',
       ' Imperial Pale Ale', ' Indian Pale Ale',
       ' VIPA, Valencia Indian Pale Ale', ' IPA (Indian Pale Ale)',
       ' European Pale Ale', ' Brown Indian Pale Ale'], dtype=object)

In [55]:
# Unificamos las India Pale Ale, excepto las que tienen otros calificativos, que son Valencia y Brown:
mask_pale_ale=df_soloartesanas["Estilo"].str.contains("Pale Ale")
mask_india=df_soloartesanas["Estilo"].str.contains("India")
mask_valencia=df_soloartesanas["Estilo"].str.contains("Valencia")
mask_brown=df_soloartesanas["Estilo"].str.contains("Brown")

# Asignamos ' India Pale Ale'
df_soloartesanas.loc[(mask_pale_ale & mask_india &~ mask_valencia &~ mask_brown ),["Estilo"]]=' India Pale Ale'

In [56]:
# También hay que unificar las tipo "American Pale Ale - APA"
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("America")]["Estilo"].value_counts()

 American Pale Ale - APA    11
 American Pale Ale           8
 American IPA                7
 American Amber Ale          6
 American Brown Ale          4
 American Golden Ale         1
 Summer American Wheat       1
 America Pale Ale            1
 American Wheat              1
Name: Estilo, dtype: int64

In [57]:
# máscaras
mask_america=df_soloartesanas["Estilo"].str.contains("America")

df_soloartesanas.loc[(mask_pale_ale & mask_america ),["Estilo"]]=' America Pale Ale'

In [58]:
# cervezas Stout están bien diferenciadas
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Stout")]["Estilo"].value_counts()

 Imperial Stout                             10
 Stout                                       8
 Russian Imperial Stout                      4
 Oatmeal Stout                               3
 Dry Stout                                   2
 Sweet Stout                                 2
 Extra Stout                                 2
 Smoke Imperial Stout                        1
 Russian Imperial Stout & Imperial Stout     1
Name: Estilo, dtype: int64

In [59]:
# las Witbier necesitan unificarse
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Wit")]["Estilo"].value_counts()

 Witbier (Belgian White)    7
 Witbier                    2
 Wit                        1
Name: Estilo, dtype: int64

In [60]:
df_soloartesanas.loc[df_soloartesanas["Estilo"].str.contains("Wit"),["Estilo"]]=' Witbier'

In [61]:
# detectamos que también hay que unificar "Weizen - Weissbier" y "Weizen-Weissbier"
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Weissbier")]["Estilo"].value_counts()

 Weizen-Weissbier             7
 Weizen - Weissbier           7
 Weizen-Weissbier y Porter    2
 Weizen Weissbier             1
 Weissbier                    1
Name: Estilo, dtype: int64

In [62]:
df_soloartesanas.loc[df_soloartesanas["Estilo"].str.contains("Weizen-Weissbier"),["Estilo"]]=' Weizen Weissbier'
df_soloartesanas.loc[df_soloartesanas["Estilo"].str.contains("Weizen - Weissbier"),["Estilo"]]=' Weizen Weissbier'

In [63]:
df_soloartesanas["Estilo"].nunique()

141

Por tanto, tenemos todas las cervezas clasificadas en un total de 144 estilos posibles.

## La fermentación
Las cervezas se clasifican en cuanto a su fermentación como Ale (alta), Lager (baja) y espontánea, aunque en esta base de datos encontramos que las cervezas vienen clasificadas en tres tipos: Ale, Lager y Mixta.
Se ha detectado que hay 31 cervezas sin clasificar,pero que pueden ser identificadas por la información sobre su estilo. La mayoría de las cervezas tienen fermentación alta y entre ellas se encuentran las IPA, weizen, wit, stout y porter. Sin embargo, las de tipo rauchbier tienen fermentación baja.

In [64]:
df_soloartesanas["Fermentación"].unique()

array(['Ale', 'Lager', nan, 'Mixta'], dtype=object)

In [65]:
print("Sin dato:",df_soloartesanas["Fermentación"].isna().sum())
df_soloartesanas["Fermentación"].value_counts()

Sin dato: 31


Ale      376
Lager     19
Mixta      1
Name: Fermentación, dtype: int64

In [66]:
# Para las que no tienen dato de fermentación ¿cuál es su estilo?
df_soloartesanas.loc[(df_soloartesanas["Fermentación"].isna()),"Estilo"].value_counts()

 Stout                                      2
 IPA                                        2
 Pale Ale                                   2
 Scotish Ale                                1
 American Brown Ale                         1
 Porter                                     1
 America Pale Ale                           1
 Belgian Golden Strong Ale                  1
 Sweet Stout                                1
 Rauchbier                                  1
 Black IPA                                  1
 Witbier                                    1
 Tropical IPA                               1
 Kölsch                                     1
 Double IPA                                 1
 Russian Imperial Stout & Imperial Stout    1
 Summer Fruit Ale                           1
 English Winter Ale                         1
 Abbey Dubbel                               1
 Weizen                                     1
 American IPA                               1
 Belgian Dubbel                   

In [67]:
# Las cervezas que tienen estilo "Ale", "IPA","Weizen","Wit","Stout" y "Porter" son de fermentación Ale
# Las tipo Rauchbier son asigno Lager

# Como solamente hay 1 Rauchbier, actualizo esa con "Lager" 
df_soloartesanas.loc[(df_soloartesanas["Estilo"].str.contains("Rauchbier") & df_soloartesanas["Fermentación"].isna()),["Fermentación"]]='Lager'

# y asigno a las demás "Ale"
df_soloartesanas.loc[(df_soloartesanas["Fermentación"].isna()),["Fermentación"]]='Ale'

# compruebo
print("Sin dato:",df_soloartesanas["Fermentación"].isna().sum())
df_soloartesanas["Fermentación"].value_counts()

Sin dato: 0


Ale      406
Lager     20
Mixta      1
Name: Fermentación, dtype: int64

Es decir, ya tenemos todas las cervezas con su tipo de fermentación.

## El amargor (IBU)
El amargor es un rasgo característico de las cervezas y muy apreciado entre los entendidos. En nuestro conjunto de cervezas artesanas vemos que hay bastante información sobre esta propiedad, pero tenemos algunos casos sin el dato. Podemos calcularlo en función de su estilo siguiendo la información existente sobre el amargor medio en función del estilo de la cerveza.
Para poder tratar este dato, primero vamos a unificarlo como float. Esto nos obliga a no tener nan en la columna, por lo que de momento vamos a usar un valor ficticio (999) para los faltantes y luego calcularemos su valor.

In [68]:
df_soloartesanas["IBU"].unique()

array([' 60 IBUs', ' 38 IBUs', nan, ' 50 IBUs', ' 20 IBUs', ' 28 IBUs',
       ' 12 IBUs', ' 10 IBUs', ' 35 IBUs', ' 26 IBUs', ' 30 IBUs',
       ' 18 IBUs', ' 70 IBUs', ' 21 IBUs', ' 40 IBUs', ' 24 IBUs',
       ' 13 IBUs', ' 44 IBUs', ' 23 IBUs', ' 31 IBUs', ' 45 IBUs',
       ' 85 IBUs', ' 65 IBUs', ' 52 IBUs', ' 100 IBUs', ' 25 IBUs',
       ' 48 IBUs', ' 22 IBUs', ' 42 IBUs', ' 14 IBUs', ' 47 IBUs',
       ' 75 IBUs', ' 27 IBUs', ' 66 IBUs', ' 49 IBUs', ' 130 IBUs',
       ' 13,8 IBUs', ' 110 IBUs', ' 37 IBUs', ' 46 IBUs', ' 17,9 IBUs',
       ' 55 IBUs', ' 32 IBUs', ' 15 IBUs', ' 39 IBUs', ' 54 IBUs',
       ' 8 IBUs', ' 19 IBUs', ' 11 IBUs', ' 80 IBUs', ' 56 IBUs',
       ' 29 IBUs', ' 120 IBUs', ' 59 IBUs', ' 23,5 IBUs', ' 36 IBUs',
       ' ND IBUs', ' 17 IBUs', ' 34 IBUs', ' 22,4 IBUs', ' 41 IBUs',
       ' 43 IBUs', ' 57,7 IBUs', ' 84 IBUs', ' 62 IBUs', ' 33 IBUs',
       ' 90 IBUs', ' 15.2 IBUs', ' 72 IBUs', ' 69 IBUs', ' 64 IBUs',
       ' 92 IBUs', ' 9,9 IBUs', ' 150 IBUs

In [69]:
df_soloartesanas["IBU"].value_counts()

 25 IBUs      27
 40 IBUs      17
 30 IBUs      16
 20 IBUs      14
 35 IBUs      12
              ..
 92 IBUs       1
 22,4 IBUs     1
 84 IBUs       1
 9,9 IBUs      1
 90 IBUs       1
Name: IBU, Length: 73, dtype: int64

In [70]:
# limpio IBUS para dejar solo los números

# Valor ficticio(999) para evitar errores con los NAN
df_soloartesanas['IBU']=df_soloartesanas['IBU'].fillna("999")

# sustituir comas por puntos
df_soloartesanas['IBU']=df_soloartesanas['IBU'].apply(lambda x: x.replace(',','.'))

# extraer el dato numérico (nums antes y después de .)
df_soloartesanas['IBU']=df_soloartesanas['IBU'].str.extract(r"(\d+(.\d+){0,1})", expand=True)[0]

# transformar en float
df_soloartesanas['IBU']=df_soloartesanas['IBU'].fillna("999")
df_soloartesanas['IBU']=df_soloartesanas['IBU'].astype('float64')
df_soloartesanas['IBU'].unique()

array([ 60. ,  38. , 999. ,  50. ,  20. ,  28. ,  12. ,  10. ,  35. ,
        26. ,  30. ,  18. ,  70. ,  21. ,  40. ,  24. ,  13. ,  44. ,
        23. ,  31. ,  45. ,  85. ,  65. ,  52. , 100. ,  25. ,  48. ,
        22. ,  42. ,  14. ,  47. ,  75. ,  27. ,  66. ,  49. , 130. ,
        13.8, 110. ,  37. ,  46. ,  17.9,  55. ,  32. ,  15. ,  39. ,
        54. ,   8. ,  19. ,  11. ,  80. ,  56. ,  29. , 120. ,  59. ,
        23.5,  36. ,  17. ,  34. ,  22.4,  41. ,  43. ,  57.7,  84. ,
        62. ,  33. ,  90. ,  15.2,  72. ,  69. ,  64. ,  92. ,   9.9,
       150. ])

In [71]:
# ¿De qué estilo son las cervezas que no tienen dato de IBU?
df_soloartesanas.loc[(df_soloartesanas["IBU"]==999),"Estilo"].value_counts()

 Pale Ale               17
 Weizen Weissbier       11
 India Pale Ale          7
 Red Ale                 6
 America Pale Ale        6
                        ..
 Steam Pilsner           1
 Wheat Ale Ecológica     1
 Trigo Premium           1
 Bávaro                  1
 Old Ale                 1
Name: Estilo, Length: 74, dtype: int64

In [72]:
# Vamos a calcular la media de IBUs de estos estilos 
df_Sin_IBU=df_soloartesanas.loc[(df_soloartesanas["IBU"]!=999),("Estilo","IBU")]
df_IBU_mean_by_Estilo=df_Sin_IBU.groupby("Estilo").mean()
df_IBU_mean_by_Estilo

,IBU
Estilo,
Abbey Dubbel,21.000000
Aged Belgian Ale,22.000000
Aged Scottish Ale,25.000000
Ale,20.750000
Altbier,36.000000
...,...
White IPA,50.000000
Winter Ale,20.000000
Winter Honey Ale (Elaborada con Miel de Romero),14.000000


Según esta consulta, tenemos 107 estilos de cervezas con sus correspondientes valores medios de IBU. Pero en total contamos con 144 estilos, es decir, hay 37 tipos de cervezas de las que no tenemos ningún dato de IBU por lo que no podemos calcular su valor mediante este procedimiento.

In [73]:
df_soloartesanas["IBU"]

1       60.0
2       38.0
3      999.0
4      999.0
5       50.0
       ...  
430    999.0
431    999.0
432     24.0
433    999.0
434    999.0
Name: IBU, Length: 427, dtype: float64

In [74]:
# volvemos a cargar los nan en los datos falsos (999) para usar fillna
df_soloartesanas["IBU"]=df_soloartesanas["IBU"].where(df_soloartesanas["IBU"]!=999.0,np.nan)

In [75]:
# asignamos la media de IBU en función del estilo a los registros con dato de IBU = 999
df_soloartesanas["IBU"] = df_soloartesanas.groupby(['Estilo'])['IBU'].transform(lambda x: x.fillna(round(x.mean(),1)))
#df_soloartesanas["IBU"] = df_soloartesanas.groupby(['Estilo'])['IBU'].transform(lambda x: x.fillna(x.mean()))
df_soloartesanas["IBU"].unique()

array([ 60. ,  38. ,  28.8,  17.1,  50. ,  20. ,  28. ,  12. ,  10. ,
        35. ,  57.8,  45.7,  26. ,  30. ,  18. ,  70. ,  21. ,  40. ,
        24. ,  13. ,  44. ,  23. ,  31. ,  45. ,  85. ,  65. ,  52. ,
       100. ,  25. ,  92. ,  48. ,  22. ,  42. ,  14. ,  47. ,  75. ,
         nan,  27. ,  66. ,  49. , 130. ,  13.8, 110. ,  37. ,  46. ,
        17.9,  33.8,  55. ,  19.6,  32. ,  15. ,  39. ,  54. ,  26.7,
        29.5,   8. ,  19. ,  11. ,  80. ,  22.8,  17.7,  65.8,  56. ,
        29. , 120. ,  59. ,  23.5,  36. ,  38.4,  17. ,  34. ,  27.3,
        22.4,  41. ,  43. ,  57.7,  31.9,  84. ,  62. ,  33. ,  55.8,
        35.1,  32.6,  90. ,  70.7,  15.2,  69.5,  43.6,  67.5,  39.5,
        72. ,  69. ,  20.8,  64. ,   9.9,  53.5, 150. ])

In [76]:
df_soloartesanas["IBU"].isna().sum()

37

Aún tenemos 40 cervezas sin valor de IBU porque no se pudo imputar la media de su estilo (posiblemente porque son las únicas cervezas de ese estilo). Ya que no tenemos suficientes registros para tratar de completar el dato mediante machine learning, optamos por eliminarlos para dejar un listado lo más completo posible.

In [77]:
df_soloartesanas.dropna(subset=['IBU'],inplace=True)
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 1 to 434
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 390 non-null    int64  
 1   color                              390 non-null    object 
 2   name                               390 non-null    object 
 3   brand                              390 non-null    object 
 4   description                        390 non-null    object 
 5   image                              390 non-null    object 
 6   Estilo                             390 non-null    object 
 7   Graduación                         377 non-null    object 
 8   Fermentación                       390 non-null    object 
 9   IBU                                390 non-null    float64
 10  Lúpulos                            91 non-null     object 
 11  European Brewing Convention (EBC)  101 non-null    object 

## Graduación o porcentaje de alcohol
Otro de los rasgos que definen a una cerveza es su grado de alcohol. Este dato viene también incompleto y con distintas representaciones no numéricas. Hay que unificarlo y tratar de obtener los datos que faltan.

In [78]:
# Modifico la columna de porcentaje de alcohol (ahora es texto) para dejar graduación como float

# Valor ficticio(999) para evitar errores con los NAN
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].fillna("999")

# sustituir comas por puntos
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].apply(lambda x: x.replace(',','.'))

# extraer el dato numérico (nums antes y después de .)
df_soloartesanas['Graduación'] = df_soloartesanas["Graduación"].str.extract(r"(\d+(.\d+){0,1})", expand=True)[0]

# transformar en float
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].astype('float64')

In [79]:
df_soloartesanas['Graduación'].unique()

array([  8.2 ,   6.  ,   5.7 ,   4.7 ,   6.6 ,   5.4 ,   4.5 ,   3.1 ,
         5.8 ,   6.7 ,   3.6 ,   5.1 ,   5.2 ,   5.5 ,   6.8 ,   5.6 ,
         6.2 ,   5.  ,   6.5 ,   6.4 ,   8.5 ,   5.3 ,   7.9 ,   4.3 ,
         8.7 ,   7.5 ,   5.9 ,   4.4 ,   4.  ,   7.2 ,   9.  ,   7.6 ,
         8.  ,   7.1 ,   4.9 ,   9.3 , 999.  ,   4.2 ,   7.  ,   4.8 ,
         4.6 ,   6.1 ,  10.1 ,   2.5 ,   6.3 ,   6.9 ,   3.5 ,   2.6 ,
         7.3 ,  14.  ,   9.5 ,   7.8 ,  10.5 ,  11.  ,  11.2 ,  10.4 ,
         5.85])

In [80]:
df_soloartesanas['Graduación'].value_counts()

5.50      39
5.00      39
4.50      27
5.20      21
6.00      20
5.80      16
4.80      16
999.00    13
6.50      13
7.00      12
5.40      12
6.20      12
4.70      12
5.60      11
7.50      11
8.00       9
6.40       7
5.30       7
5.10       6
4.00       6
8.50       6
4.20       5
6.70       5
4.90       4
6.10       4
4.30       4
6.60       4
9.00       4
6.80       4
6.30       3
5.70       3
7.20       3
4.60       3
8.20       2
5.90       2
7.10       2
4.40       2
3.50       2
14.00      1
10.10      1
7.30       1
2.50       1
7.90       1
9.50       1
10.50      1
11.00      1
6.90       1
10.40      1
5.85       1
3.10       1
7.80       1
3.60       1
2.60       1
8.70       1
7.60       1
11.20      1
9.30       1
Name: Graduación, dtype: int64

In [81]:
# ¿Cómo son las cervezas que no tienen dato de graducación?
df_soloartesanas[df_soloartesanas['Graduación']==999.00]

,id,color,name,brand,description,image,Estilo,Graduación,Fermentación,IBU,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,Población,Provincia,Comunidad Autónoma,País
88,89,rubia,Naparbier Insider,(cervezas artesanas Naparbier),\nInsider es una IPA anaranjada elaborada por ...,https://static2.soloartesanas.es/2722-large_de...,India Pale Ale,999.0,Ale,57.8,NaN,NaN,NaN,NaN,Cebada,Mutilva Baja,Navarra,Navarra,España
97,98,rubia,Dougall's Happy Otter,(cervezas artesanas Dougall's),\nCerveza de temporada de Dougalls. Una Pale A...,https://static3.soloartesanas.es/1719-large_de...,Pale Ale,999.0,Ale,28.8,NaN,NaN,NaN,NaN,Cebada,Liérganes,Cantabria,Cantabria,España
145,146,rubia,Vier IPA,(cervezas artesanas Cervezas Vier),\nAnte tienes una cerveza artesana de estilo I...,https://static3.soloartesanas.es/1006-large_de...,India Pale Ale,999.0,Ale,57.8,NaN,NaN,NaN,8-12ºC,Cebada,Valladolid,Valladolid,Castilla y León,España
172,173,rubia,Arriaca Trigo,(cervezas artesanas Cervezas Arriaca),\nArrica Trigo es una de las cervezas artesana...,https://static2.soloartesanas.es/3440-large_de...,Weizen Weissbier,999.0,Ale,17.1,NaN,NaN,NaN,NaN,Trigo,Yunquera de Henares,Guadalajara,Castilla-La Mancha,España
197,198,rubia,Mesopotamia Pale Ale,(cervezas artesanas Abadía de Aribayos),\nLa cerveza artesana Mesopotamia Pale Ale que...,https://static2.soloartesanas.es/702-large_def...,Cream Ale,999.0,Ale,47.0,NaN,NaN,NaN,NaN,Cebada,Moraleja del Vino,Zamora,Andalucía,España
279,280,tostada,Odiel Tostada de trigo,(cervezas artesanas Cerveza Odiel),\nUna cerveza tostada de trigo que no se ciñe ...,https://static2.soloartesanas.es/2243-large_de...,Düsseldorf Altbier,999.0,Ale,37.0,NaN,NaN,NaN,NaN,Trigo,Valverde del Camino,Huelva,Andalucía,España
293,294,tostada,Pedraforca Torrada,(cervezas artesanas La Cervesera del Pedraforca),"\nUna cerveza artesana con aroma, cuerpo y el ...",https://static1.soloartesanas.es/1490-large_de...,Brown Ale,999.0,Ale,30.0,NaN,NaN,NaN,NaN,Cebada,Saldes,Barcelona,Cataluña,España
353,354,negra,Arriaca Imperial Russian Stout,(cervezas artesanas Cervezas Arriaca),\nUna cerveza negra de tipo Ale con una gradua...,https://static3.soloartesanas.es/3446-large_de...,Russian Imperial Stout,999.0,Ale,70.7,NaN,NaN,NaN,NaN,NaN,Yunquera de Henares,Guadalajara,Castilla-La Mancha,España
376,377,negra,Quer Pacific Gem,(cervezas artesanas Quer Beer),\nPacific Gem es la cerveza artesana negra de ...,https://static1.soloartesanas.es/990-large_def...,Stout,999.0,Ale,33.0,NaN,40-45,"Pale, Chocolate, Carafa, Roasted Barley, Caram...",NaN,Cebada,Berga,Barcelona,Cataluña,España
386,387,negra,Arriaca Porter,(cervezas artesanas Cervezas Arriaca),\nArrica Porter es una de las cervezas artesan...,https://static1.soloartesanas.es/3442-large_de...,Imperial Stout,999.0,Ale,69.5,NaN,NaN,NaN,NaN,Cebada,Yunquera de Henares,Guadalajara,Castilla-La Mancha,España


In [82]:
#.             ----------      SEGUIR POR AQUÍ!!!

In [83]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 1 to 434
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 390 non-null    int64  
 1   color                              390 non-null    object 
 2   name                               390 non-null    object 
 3   brand                              390 non-null    object 
 4   description                        390 non-null    object 
 5   image                              390 non-null    object 
 6   Estilo                             390 non-null    object 
 7   Graduación                         390 non-null    float64
 8   Fermentación                       390 non-null    object 
 9   IBU                                390 non-null    float64
 10  Lúpulos                            91 non-null     object 
 11  European Brewing Convention (EBC)  101 non-null    object 

### Características que están completas
Veamos cómo son las características que no tienen datos nulos, como son el color, nombre, marca...


In [84]:
df_soloartesanas['brand'].value_counts()

(cervezas artesanas Birra & Blues)        13
(cervezas artesanas Cervezas Arriaca)     11
(cervezas artesanas Althaia Artesana)      9
(cervezas artesanas Dawat)                 9
(cervezas artesanas Sagra)                 7
                                          ..
(cervezas artesanas Pato Nero)             1
(cervezas artesanas Del Pueblo)            1
(cervezas artesanas Er Boquerón)           1
(cervezas artesanas Cerveza del Jerte)     1
(cervezas artesanas Cervezas Nel)          1
Name: brand, Length: 139, dtype: int64

In [85]:
# eliminar "cervezas artesanas " y los paréntesis en el campo "brand"
df_soloartesanas['brand']=df_soloartesanas['brand'].apply(lambda x: x.replace('(cervezas artesanas ',''))
df_soloartesanas['brand']=df_soloartesanas['brand'].apply(lambda x: x.replace(')',''))

In [86]:
# ejemplo de una cerveza
df_soloartesanas.iloc[223]

id                                                                                 252
color                                                                            rubia
name                                                                              Clot
brand                                                                         Birra 08
description                          \nCerveza Clot, así se llama la cerveza artesa...
image                                https://static3.soloartesanas.es/143-large_def...
Estilo                                                                        Pale Ale
Graduación                                                                         4.3
Fermentación                                                                       Ale
IBU                                                                                 35
Lúpulos                                                                        Fuggles
European Brewing Convention (EBC)          

In [87]:
df_soloartesanas.columns

Index(['id', 'color', 'name', 'brand', 'description', 'image', 'Estilo',
       'Graduación', 'Fermentación', 'IBU', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población', 'Provincia', 'Comunidad Autónoma', 'País'],
      dtype='object')

In [88]:
df_soloartesanas["color"].unique()

array(['rubia', 'tostada', 'negra', 'blanca', 'roja'], dtype=object)

In [89]:
df_soloartesanas["name"].unique()

array(['Alegría & Boris Brew Ipanosuarus Rex', 'Roy The Bull Mango',
       'Marijuana', 'Arriaca Trigo (lata)', 'Bertus White IPA',
       'Bertus Summer Ale', 'Bertus Carpe Diem', '90 Varas La Mala',
       '90 Varas Zerezo', 'Althaia Rabosa', 'Mica Raíz',
       'Arriaca Session IPA (lata)', 'OOB Hoppy Blonde',
       'Familia Serra Maula', 'Familia Serra Butoni',
       'Familia Serra Mussa', 'Sargs IPA', 'Birra & Blues Mago de Oz',
       'Laugar Basurde', 'Laugar EPA!', 'Birra & Blues Smoke my Beer',
       'Birra & Blues Amber', 'Birra & Blues IPA Blues',
       'Birra & Blues Tripel', 'Birra & Blues La Rubia',
       'Birra & Blues Doble Malta', 'Madriz Chueca', 'Alma Turdetana',
       'La Virgen IPA', 'La Quarantamaula American Pale Ale',
       'La Quarantamaula American Golden Ale', '3Monos Vito Hopleone',
       '3Monos Dirty Harry', '3Monos Monkey Business',
       '3Monos Golden Ape', 'Skunk Diaple',
       'The Flying Inn Liquid Nemesis',
       'The Flying Inn Telepath

In [90]:
df_soloartesanas["brand"].unique()

array(['Cervezas Alegría', 'Roy The Bull', 'Marijuana',
       'Cervezas Arriaca', 'Bertus', '90 Varas', 'Althaia Artesana',
       'Cervezas Mica', 'OOB', 'Familia Serra', 'Sargs', 'Birra & Blues',
       'Laugar', 'Madriz', 'Alma Turdetana', 'Cerveza La Virgen',
       'La Quarantamaula', '3Monos', 'Ordio Minero', 'The Flying Inn',
       'Fundación Badalona Capaç', 'Cervecera Libre', 'Cerveza Ballut',
       'Castelló Beer Factory', 'Dawat', "L'anjub", 'Cervesa Marina',
       'Quana', 'Panda Beer', 'Garagart', 'Ausesken', 'Del Pueblo',
       'Bonvivant', 'La Calavera', 'Arévaka', 'Tyris', 'Cervezas Quijota',
       'Cervezas Abadía', 'Morlaco Beer', 'Kadabra', 'Espina de Ferro',
       'Pato Nero', 'Cañonita', 'Cerveza Tartessos',
       'As Cervesa Artesana', 'Cerveza Odiel', 'Cerveses La Pirata',
       'Naparbier', 'Mateo & Bernabé', 'Nurse', 'Cervezas Speranto',
       'Goose', 'Smach', 'Cervezas Libre', "Dougall's", 'Drunken Bros',
       'Beauty', 'Cartujana', 'Zeta Beer', '

### Columnas con poca información

Del resto de columnas vemos que faltan también muchos datos en 'Lúpulos', 'European Brewing Convention (EBC)', 'Maltas' y 'Temperatura de consumo'. El dato del tipo de grano no está completo pero podría resultar interesante de modo que por ahora lo vamos a mantener.

In [91]:
df_soloartesanas.drop(columns=[ 'Lúpulos', 'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo'], inplace=True)

In [92]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 1 to 434
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  390 non-null    int64  
 1   color               390 non-null    object 
 2   name                390 non-null    object 
 3   brand               390 non-null    object 
 4   description         390 non-null    object 
 5   image               390 non-null    object 
 6   Estilo              390 non-null    object 
 7   Graduación          390 non-null    float64
 8   Fermentación        390 non-null    object 
 9   IBU                 390 non-null    float64
 10  Grano               280 non-null    object 
 11  Población           390 non-null    object 
 12  Provincia           390 non-null    object 
 13  Comunidad Autónoma  390 non-null    object 
 14  País                390 non-null    object 
dtypes: float64(2), int64(1), object(12)
memory usage: 48.8+ K

### Unificamos el tipo de grano
A pesar de que este dato falta en bastantes registros, lo dejamos porque puede ser interesante para pintar. Para ello, primero unificamos los datos ya que algunos están escritos de varias formas.

In [93]:
df_soloartesanas["Grano"].unique()

array([nan, 'Cebada', 'Trigo', 'Avena', 'Arroz', 'Cebada y trigo',
       'Trigo y cebada (50%)', 'Cebada y Trigo', 'Cebada y avena'],
      dtype=object)

In [94]:
df_soloartesanas["Grano"].value_counts()

Cebada                  231
Trigo                    38
Avena                     4
Trigo y cebada (50%)      2
Cebada y Trigo            2
Cebada y avena            1
Cebada y trigo            1
Arroz                     1
Name: Grano, dtype: int64

In [95]:
# Unifico cebada y trigo
df_soloartesanas.loc[df_soloartesanas["Grano"]=='Trigo y cebada (50%)',"Grano"]='Cebada y trigo'
df_soloartesanas.loc[df_soloartesanas["Grano"]=='Cebada y Trigo',"Grano"]='Cebada y trigo'

In [96]:
df_soloartesanas["Grano"].value_counts()

Cebada            231
Trigo              38
Cebada y trigo      5
Avena               4
Cebada y avena      1
Arroz               1
Name: Grano, dtype: int64

In [97]:
df_soloartesanas.to_csv("data/soloartesanas_limpio.csv", sep = ';', index = False)

In [98]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 390 entries, 1 to 434
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  390 non-null    int64  
 1   color               390 non-null    object 
 2   name                390 non-null    object 
 3   brand               390 non-null    object 
 4   description         390 non-null    object 
 5   image               390 non-null    object 
 6   Estilo              390 non-null    object 
 7   Graduación          390 non-null    float64
 8   Fermentación        390 non-null    object 
 9   IBU                 390 non-null    float64
 10  Grano               280 non-null    object 
 11  Población           390 non-null    object 
 12  Provincia           390 non-null    object 
 13  Comunidad Autónoma  390 non-null    object 
 14  País                390 non-null    object 
dtypes: float64(2), int64(1), object(12)
memory usage: 48.8+ K